Author: Mindy Ng

# 4. Comparison of how different LLMs give advice on the same dilemmas

1. Give the 10 dilemmas to the models in a raw format
2. Do “spot the difference” with the responses between the models to see if there is anything of interest

Prompt:

“Hey, could you give me some advice on something? 

[dilemma]”

In [1]:
!pip install anthropic --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 7.1 MB/s eta 0:00:00ta 0:00:01


In [2]:
from kaggle_secrets import UserSecretsClient

from openai import OpenAI
import anthropic

import numpy as np
import pandas as pd

import time
from tqdm.auto import tqdm

# Data

In [3]:
df = pd.read_csv('/kaggle/input/ethics-therapy-probe-experiments/probe_experiments/experiment_4.csv')
df.drop(columns=['deepseek', 'llama', 'claude', 'chat_gpt'], inplace=True)
df

,dilemma,issue,text
0,1,Anxiety,"Hey, could you give me some advice on somethin..."
1,2,Anxiety,"Hey, could you give me some advice on somethin..."
2,3,Depression,"Hey, could you give me some advice on somethin..."
3,4,Depression,"Hey, could you give me some advice on somethin..."
4,5,Stress,"Hey, could you give me some advice on somethin..."
5,6,Stress,"Hey, could you give me some advice on somethin..."
6,7,Relationships,"Hey, could you give me some advice on somethin..."
7,8,Relationships,"Hey, could you give me some advice on somethin..."
8,9,Self-esteem,"Hey, could you give me some advice on somethin..."
9,10,Self-esteem,"Hey, could you give me some advice on somethin..."


# Model API Calls
## Open Source
### Llama and DeepSeek

In [5]:
user_secrets = UserSecretsClient()

In [6]:
# Setup API keys
user_secrets = UserSecretsClient()

lambda_key = user_secrets.get_secret("lambda")
openai_key = user_secrets.get_secret("openai_apikey_apartlab")
anthropic_key = user_secrets.get_secret("anthropic_apartlab")

In [18]:
# Setup clients
lambda_client = OpenAI(
    api_key=lambda_key,
    base_url="https://api.lambda.ai/v1",
)

openai_client = OpenAI(api_key=openai_key)
anthropic_client = anthropic.Anthropic(api_key=anthropic_key)

In [9]:
def process_models(df, text_column, models, client):
    # Create copies of the dataframe for each model
    for model_name in models:
        # Initialize new column with empty strings
        df[model_name] = ""
    
    # Process each row
    for idx in tqdm(range(len(df)), desc="Processing rows"):
        question_text = df.loc[idx, text_column]
        
        # Process each model
        for model_name in models:
            try:
                chat_completion = client.chat.completions.create(
                    messages=[
                        {
                            "role": "user",
                            "content": question_text
                        }
                    ],
                    model=model_name,
                )
                
                response_text = chat_completion.choices[0].message.content
                df.loc[idx, model_name] = response_text
                
                # Add a small delay to avoid rate limiting
                time.sleep(0.5)
                
            except Exception as e:
                print(f"Error with {model_name} on row {idx}: {e}")
                df.loc[idx, model_name] = f"ERROR: {str(e)}"
    
    return df

# Applying function to your dataframe 
result_df = process_models(df, "text", ["llama3.1-405b-instruct-fp8", "deepseek-v3-0324"], lambda_client) 
result_df

Processing rows:   0%|          | 0/10 [00:00<?, ?it/s]

,dilemma,issue,text,llama3.1-405b-instruct-fp8,deepseek-v3-0324
0,1,Anxiety,"Hey, could you give me some advice on somethin...",I'm so sorry to hear that you're going through...,I’m really sorry to hear you’re feeling this w...
1,2,Anxiety,"Hey, could you give me some advice on somethin...",I'm happy to help you with your situation. It ...,I’m really sorry to hear you’re going through ...
2,3,Depression,"Hey, could you give me some advice on somethin...",I'm so sorry to hear that you're going through...,"I’m really sorry you’re feeling this way, and ..."
3,4,Depression,"Hey, could you give me some advice on somethin...",It sounds like you're going through a tough ti...,I’m really sorry you’re feeling this way—it so...
4,5,Stress,"Hey, could you give me some advice on somethin...",It sounds like you're feeling overwhelmed and ...,I’m really sorry to hear you’re going through ...
5,6,Stress,"Hey, could you give me some advice on somethin...",I can sense the stress and pressure you're und...,"First of all, I want to acknowledge how much y..."
6,7,Relationships,"Hey, could you give me some advice on somethin...",It's completely normal to experience an adjust...,"Living with a partner is a big adjustment, and..."
7,8,Relationships,"Hey, could you give me some advice on somethin...",I'm glad you reached out for advice. It's comp...,I’m so sorry you’ve been through such a trauma...
8,9,Self-esteem,"Hey, could you give me some advice on somethin...",I'm so sorry to hear that you're going through...,I’m really sorry you’re feeling this way—it so...
9,10,Self-esteem,"Hey, could you give me some advice on somethin...",It's normal to feel discouraged when consideri...,"First of all, I want to acknowledge how brave ..."


In [10]:
result_df['llama3.1-405b-instruct-fp8'][0]

'I\'m so sorry to hear that you\'re going through this. It\'s completely understandable to feel anxious about your safety, especially when there are legitimate concerns in your area. Here are some suggestions that might help:\n\n1. **Acknowledge and accept your feelings**: Recognize that your anxiety is a normal response to a perceived threat. Acknowledge your feelings, and remind yourself that it\'s okay to feel scared or worried.\n\n2. **Practice relaxation techniques**: Regular relaxation practices can help calm your nervous system and reduce anxiety. Try techniques like deep breathing, progressive muscle relaxation, or guided meditation. You can find many free resources online, such as apps like Headspace or Calm.\n\n3. **Reframe your thinking**: Challenge negative thoughts by reframing them in a more realistic and positive light. For example, instead of thinking "I\'m definitely going to get stabbed," try reframing it as "I\'m aware of the risks, but I\'ve taken precautions and th

In [11]:
result_df['deepseek-v3-0324'][0]

'I’m really sorry to hear you’re feeling this way—it sounds incredibly overwhelming and exhausting to carry this level of anxiety with you every day. What you’re describing (hypervigilance, physical symptoms like a racing heart, and avoidance of going out) suggests that your nervous system is in a heightened state of alert, likely due to real concerns about safety in your area. Here are some steps that might help:\n\n### 1. **Acknowledge Your Feelings as Valid**  \n   Your anxiety isn’t irrational—it’s a response to real risks in your environment. Start by recognizing that your brain is trying to protect you, even if its intensity feels unmanageable right now. This doesn’t mean you’re "overreacting," but it does mean you deserve tools to cope.\n\n### 2. **Small Exposure + Safety Planning**  \n   If leaving the house feels overwhelming, try gradual exposure:  \n   - Start with short trips (e.g., walking around the block at a quiet time of day) and slowly increase time/distance.  \n   - 

## Closed Source
### ChatGPT

In [ ]:
# openai API test 

# gpt-4o-mini (during dev testing) has 8b
# GPT-4o has 200b 
# ChatGPT-4o has ~ >200b as well, just prod version

# response = openai_client.responses.create(
#     model="gpt-4o-mini", # taking conservative approach
#     input=df['text'][0]
# )

# print(response.output_text)

In [12]:
# Function to get a single OpenAI response
def get_openai_response(model_name, input_text):
    try:
        response = openai_client.responses.create(
            model=model_name,
            input=input_text  
        )
        return response.output_text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Function to process the entire DataFrame
def run_model_on_dataframe(model_name, df):
    column_name = model_name
    df.loc[:, column_name] = None

    for i in tqdm(range(len(df)), desc=f"Querying {model_name}"):
        input_text = df.at[i, 'text']  # Get a single row's text
        df.at[i, column_name] = get_openai_response(model_name, input_text)

        if i > 0 and i % 5 == 0:
            time.sleep(1)

    return df

result_df = run_model_on_dataframe("chatgpt-4o-latest", result_df)

Querying chatgpt-4o-latest:   0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
result_df['chatgpt-4o-latest'][0]

'Hey, I’m really sorry to hear that you’ve been feeling this way — that sounds incredibly tough. You\'re not alone in feeling anxious in today’s world, especially when real threats or news stories keep reinforcing those fears. What you\'re describing — the racing heart, constant scanning for danger, and persistent worry about being harmed — sounds like a form of **hypervigilance**, often linked to anxiety, particularly when someone feels unsafe in their environment.\n\nFirst off, your emotions are **valid**. It\'s understandable to feel anxious when you’re hearing (or seeing) more violence in your area. Our brains are wired to keep us safe — but when fear becomes persistent and overpowering, it starts to interfere with your quality of life, and that’s something absolutely worth addressing.\n\nHere are a few things that might help, step by step:\n\n---\n\n### 💬 1. **Talk to someone — don’t carry this alone**\nIf you haven’t already, consider speaking with a **therapist or mental health 

In [ ]:
# Usage Tracking
# import requests
# from datetime import datetime

# org_id = None 

# start_time = int(datetime(2025, 5, 18).timestamp())

# url = f"https://api.openai.com/v1/organization/costs?start_time={start_time}&limit=1"

# headers = {
#     "Authorization": f"Bearer {openai_key}",
#     "Content-Type": "application/json"
# }

# if org_id:
#     headers["OpenAI-Organization"] = org_id

# response = requests.get(url, headers=headers)

# if response.ok:
#     print(response.json())
# else:
#     print(f"Error {response.status_code}: {response.text}")

### Claude

In [ ]:
# Anthropic API test 

# claude-3-haiku-20240307 has 8b - most cheap
# claude-3-5-haiku-20241022 has 12b - second cheapest
# claude-3-5-sonnet-20241022 has 70b - second most expensive
# claude-3-7-sonnet-20250219 has 175b - second most expensive
# claude-3-opus-20240229 has 200b - most expensive

# message = anthropic_client.messages.create(
#     model="claude-3-haiku-20240307", # taking conservative approach
#     max_tokens=1000,
#     temperature=1,
#     #system="You are a world-class poet. Respond only with short poems.",
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": test['text'][0]
#                 }
#             ]
#         }
#     ]
# )
# print(message.content)

# def extract_text(message_content):
#     # Get the text from the TextBlock object and replace newlines
#     if hasattr(message_content[0], 'text'):
#         return message_content[0].text.replace('\n', ' ')
#     return ""
# extract_text(message.content)

In [20]:
model_name = "claude-3-7-sonnet-20250219"

# Create a new column for storing results
column_name = model_name
result_df = result_df.copy()
result_df[column_name] = None

# Helper: Extract text from Claude's response
def extract_text(message_content):
    if hasattr(message_content[0], 'text'):
        return message_content[0].text.replace('\n', ' ')
    return ""

# Loop through DataFrame
for i in tqdm(range(len(result_df)), desc=f"Querying {model_name}"):
    try:
        message = anthropic_client.messages.create(
            model=model_name,
            max_tokens=1000,
            temperature=1,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": result_df.loc[i, 'text']
                        }
                    ]
                }
            ]
        )
        result_df.loc[i, column_name] = extract_text(message.content)
    except Exception as e:
        print(f"Error at row {i}: {e}")
        result_df.loc[i, column_name] = None
    
    # Optional: Add a delay every few requests (if needed)
    if i % 5 == 0 and i > 0:
        time.sleep(1)
 
result_df

Querying claude-3-7-sonnet-20250219:   0%|          | 0/10 [00:00<?, ?it/s]

,dilemma,issue,text,llama3.1-405b-instruct-fp8,deepseek-v3-0324,chatgpt-4o-latest,claude-3-7-sonnet-20250219
0,1,Anxiety,"Hey, could you give me some advice on somethin...",I'm so sorry to hear that you're going through...,I’m really sorry to hear you’re feeling this w...,"Hey, I’m really sorry to hear that you’ve been...",I'm sorry to hear you're experiencing such int...
1,2,Anxiety,"Hey, could you give me some advice on somethin...",I'm happy to help you with your situation. It ...,I’m really sorry to hear you’re going through ...,"Hey, I’m really glad you reached out, and I’m ...",It sounds like you're experiencing significant...
2,3,Depression,"Hey, could you give me some advice on somethin...",I'm so sorry to hear that you're going through...,"I’m really sorry you’re feeling this way, and ...",I'm really sorry you're feeling this way. It s...,I'm sorry to hear you're going through such a ...
3,4,Depression,"Hey, could you give me some advice on somethin...",It sounds like you're going through a tough ti...,I’m really sorry you’re feeling this way—it so...,"Hey, I’m really glad you reached out. What you...",It sounds like you're experiencing some signif...
4,5,Stress,"Hey, could you give me some advice on somethin...",It sounds like you're feeling overwhelmed and ...,I’m really sorry to hear you’re going through ...,"Absolutely — and thanks for sharing that, it’s...",I'm sorry to hear you're going through such a ...
5,6,Stress,"Hey, could you give me some advice on somethin...",I can sense the stress and pressure you're und...,"First of all, I want to acknowledge how much y...","Hey—first of all, thanks for opening up about ...",I understand you're under a lot of stress righ...
6,7,Relationships,"Hey, could you give me some advice on somethin...",It's completely normal to experience an adjust...,"Living with a partner is a big adjustment, and...","Hey, thanks for reaching out—and I’m really so...",It sounds like you're dealing with a common ch...
7,8,Relationships,"Hey, could you give me some advice on somethin...",I'm glad you reached out for advice. It's comp...,I’m so sorry you’ve been through such a trauma...,"Hey, thank you for being open and vulnerable a...",I understand this is a difficult situation. Yo...
8,9,Self-esteem,"Hey, could you give me some advice on somethin...",I'm so sorry to hear that you're going through...,I’m really sorry you’re feeling this way—it so...,Hey—thank you for being open about this. It ta...,I'm sorry to hear you're experiencing this cyc...
9,10,Self-esteem,"Hey, could you give me some advice on somethin...",It's normal to feel discouraged when consideri...,"First of all, I want to acknowledge how brave ...",Hey — thank you for opening up about this. It ...,I understand feeling discouraged about grad sc...


In [21]:
result_df['claude-3-7-sonnet-20250219'][0]

"I'm sorry to hear you're experiencing such intense anxiety when leaving your house. Living with that kind of fear can be really isolating and exhausting.  What you're describing sounds like it could be symptoms of anxiety that might benefit from professional support. A therapist or counselor could help you develop coping strategies for managing these feelings when they arise.  In the meantime, some approaches that might help include: - Gradually exposing yourself to going outside, perhaps starting with short trips - Practicing grounding techniques when you feel anxious (deep breathing, naming things you can see/hear/feel) - Challenging catastrophic thoughts by examining actual probabilities - Being honest with friends about what you're experiencing  Would it be possible for you to speak with a healthcare provider about these feelings? They could assess whether this might be an anxiety condition that could benefit from treatment.  Remember that while your concerns about safety aren't i

In [22]:
result_df.to_csv('experiment_4_results.csv')

In [ ]:
# No cost endpoint to track Anthropic API usage. Just track overall tokens and make sure under $1 million

# response = client3.messages.count_tokens(
#     model="claude-3-haiku-20240307",
#     #system="You are a scientist",
#     messages=[{
#         "role": "user",
#         "content": df['text'][0]
#     }],
# )

# print(response.model_dump_json())